# Task B: RFQ Similarity

In this task, I’m calculating how similar different RFQs are to each other based on their properties, both numeric and categorical, so that we can find the top matches for any given RFQ. Here’s a breakdown of what I did in each step:

## Load Datasets

I load the RFQs and the reference properties files. This is just to get all the data ready so I can work with it. I also checked the columns to make sure the names are what I expect.

In [1]:
import pandas as pd
import numpy as np


# Load datasets
rfq = pd.read_csv("../data/rfq.csv")              # RFQs
ref = pd.read_csv("../data/reference_properties.tsv", sep='\t')  # Reference grade

# Inspect columns to confirm names
print(rfq.columns)
print(ref.columns)

Index(['id', 'grade', 'grade_suffix', 'coating', 'finish', 'surface_type',
       'surface_protection', 'form', 'thickness_min', 'thickness_max',
       'width_min', 'width_max', 'length_min', 'height_min', 'height_max',
       'weight_min', 'weight_max', 'inner_diameter_min', 'inner_diameter_max',
       'outer_diameter_min', 'outer_diameter_max', 'yield_strength_min',
       'yield_strength_max', 'tensile_strength_min', 'tensile_strength_max'],
      dtype='object')
Index(['Grade/Material', 'UNS_No', 'Steel_No', 'Standards', 'Carbon (C)',
       'Manganese (Mn)', 'Silicon (Si)', 'Sulfur (S)', 'Phosphorus (P)',
       'Chromium (Cr)', 'Nickel (Ni)', 'Molybdenum (Mo)', 'Vanadium (V)',
       'Tungsten (W)', 'Cobalt (Co)', 'Copper (Cu)', 'Aluminum (Al)',
       'Titanium (Ti)', 'Niobium (Nb)', 'Boron (B)', 'Nitrogen (N)',
       'Tensile strength (Rm)', 'Yield strength (Re or Rp0.2)',
       'Elongation (A%)', 'Reduction of area (Z%)', 'Hardness (HB, HV, HRC)',
       'Impact toughness 

## Task B.1 — Reference join & missing values

Here, I normalized the grade names in both datasets so that they match (lowercase, stripped of spaces). Then I parsed numeric ranges in the reference properties (like “2.5–3.0” or “≤5”) into min, max, and mid values. Finally, I joined the RFQs with the reference properties based on the normalized grades and flaged any missing references.

In [2]:
# Normalize RFQ grades
rfq['grade_norm'] = rfq['grade'].str.lower().str.strip()

# Normalize reference grades
# Adjust 'Grade/Material' based on actual column
ref['grade_norm'] = ref['Grade/Material'].str.lower().str.strip()

In [3]:
import re
def parse_range(s):
    if pd.isna(s):
        return (np.nan, np.nan)
    s = str(s).replace('≤','').replace('≥','').replace('MPa','').strip()
    # Split by - or en dash
    parts = re.split(r'[-–]', s)
    try:
        a = float(parts[0].replace(',', '.')) if parts[0] else np.nan
        b = float(parts[1].replace(',', '.')) if len(parts)>1 and parts[1] else a
        return (min(a,b), max(a,b))
    except:
        return (np.nan, np.nan)

# Example numeric columns to parse
numeric_cols = [
    'Carbon (C)','Manganese (Mn)','Silicon (Si)','Sulfur (S)','Phosphorus (P)',
    'Tensile strength (Rm)','Yield strength (Re or Rp0.2)'
]

for col in numeric_cols:
    if col in ref.columns:
        ref[[f'{col}_min', f'{col}_max']] = ref[col].apply(lambda x: pd.Series(parse_range(x)))
        ref[f'{col}_mid'] = ref[[f'{col}_min', f'{col}_max']].mean(axis=1)

### Join RFQs with reference

In [4]:
# Merge RFQs with reference properties
rfq_ref = rfq.merge(ref, on='grade_norm', how='left')

# Flag missing reference grades
rfq_ref['missing_ref'] = rfq_ref['grade_norm'].isnull() | rfq_ref['grade_norm'].isin(ref['grade_norm'].isnull())

## Task B.2 — Feature engineering
In this step, I prepared the features for similarity calculation:

For numeric dimensions, I made sure we have min and max columns, and if max is missing I filled it with the min.

I defined the categorical columns that we want to compare.

I also created midpoints for numeric grades so that they can be used in similarity calculations.

In [5]:
# Dimensions as intervals
dimensions = ['thickness','width','length','height','weight',
              'inner_diameter','outer_diameter',
              'yield_strength','tensile_strength']

for dim in dimensions:
    min_col = f'{dim}_min'
    max_col = f'{dim}_max'

    # Create the columns if they don't exist
    if min_col not in rfq_ref.columns:
        rfq_ref[min_col] = np.nan
    if max_col not in rfq_ref.columns:
        rfq_ref[max_col] = np.nan

    # Fill missing max with min (singleton case)
    rfq_ref[max_col] = rfq_ref[max_col].fillna(rfq_ref[min_col])

In [6]:
# Define categorical similarity
categorical_cols = ['coating','finish','form','surface_type']

In [7]:
# Grade midpoints
grade_numeric_cols = [f'{col}_mid' for col in numeric_cols if f'{col}_mid' in rfq_ref.columns]

## Task B.3 — Similarity calculation

I created functions to compute similarity between two RFQs:

Dimension similarity: I used an interval-based IoU, which looks at how much two numeric ranges overlap.

Categorical similarity: I checked if the categorical properties are the same.

Grade similarity: For numeric grade properties, I compared the values using a simple ratio of smaller to larger.

Then I combined all three types of similarity into an overall score using weights (dimension 0.5, categorical 0.2, grade 0.3).

In [8]:
# Interval IoU
def interval_iou(a_min, a_max, b_min, b_max):
    """Compute IoU (intersection over union) for numeric intervals."""
    if np.isnan(a_min) or np.isnan(a_max) or np.isnan(b_min) or np.isnan(b_max):
        return np.nan
    inter = max(0, min(a_max, b_max) - max(a_min, b_min))
    union = max(a_max, b_max) - min(a_min, b_min)
    return inter / union if union > 0 else 0

In [9]:
# RFQ similarity function
def rfq_similarity(a, b, dimension_list, categorical_list, grade_numeric_cols):
    """Compute overall similarity between two RFQs."""
    
    #Dimension similarity
    dim_sims = []
    for dim in dimension_list:
        min_col = f'{dim}_min'
        max_col = f'{dim}_max'

        # Only compute if both RFQs have the dimension
        if all(col in a.index for col in [min_col, max_col]) and all(col in b.index for col in [min_col, max_col]):
            sim = interval_iou(a[min_col], a[max_col], b[min_col], b[max_col])
            if sim is not None and not np.isnan(sim):
                dim_sims.append(sim)

    dim_score = np.mean(dim_sims) if dim_sims else 0

    #Categorical similarity
    cat_sims = []
    for cat in categorical_list:
        if cat in a.index and cat in b.index:
            val_a, val_b = a[cat], b[cat]
            if pd.isna(val_a) or pd.isna(val_b):
                continue
            cat_sims.append(1 if val_a == val_b else 0)

    cat_score = np.mean(cat_sims) if cat_sims else 0

    #Grade numeric properties similarity
    grade_sims = []
    for col in grade_numeric_cols:
        if col in a.index and col in b.index:
            val_a, val_b = a[col], b[col]
            if pd.isna(val_a) or pd.isna(val_b):
                continue
            # Simple similarity: ratio of smaller/larger
            grade_sims.append(min(val_a, val_b) / max(val_a, val_b) if max(val_a, val_b) != 0 else 0)

    grade_score = np.mean(grade_sims) if grade_sims else 0

    # Aggregate similarity
    # Example: weighted average 
    weights = {
        'dimension': 0.5,
        'categorical': 0.2,
        'grade': 0.3
    }

    overall_score = (
        dim_score * weights['dimension'] +
        cat_score * weights['categorical'] +
        grade_score * weights['grade']
    )

    return overall_score

## Compute top-3 most similar RFQs

I looped through all RFQs and compared each one with every other RFQ using the similarity function. For each RFQ, I picked the top 3 most similar ones and saved them into a DataFrame. This gave me an easy way to see which RFQs are closest to any given RFQ.

Finally, I saved the results to a CSV.

In [10]:
from tqdm import tqdm  # optional, shows progress bar

# Define the columns to use (replace with your dataset columns)
dimension_list = [
    'thickness', 'width', 'length', 'height',
    'weight', 'inner_diameter', 'outer_diameter',
    'yield_strength', 'tensile_strength'
]
categorical_list = ['coating', 'finish', 'form', 'surface_type']
grade_numeric_cols = [
    'C', 'Mn', 'Si', 'S', 'P', 'Cr', 'Ni', 'Mo', 'V', 'W', 'Co', 'Cu', 'Al', 'Ti', 'Nb', 'B', 'N', 
    'tensile_strength_mid', 'yield_strength_mid'
]

top3_list = []

# Use tqdm to monitor progress if dataset is large
for idx_a, row_a in tqdm(rfq_ref.iterrows(), total=len(rfq_ref), desc="Processing RFQs"):
    sims = []
    for idx_b, row_b in rfq_ref.iterrows():
        if row_a['id'] == row_b['id']:
            continue
        score = rfq_similarity(row_a, row_b, dimension_list, categorical_list, grade_numeric_cols)
        sims.append((row_b['id'], score))
    
    # Get top 3 matches
    top3 = sorted(sims, key=lambda x: x[1], reverse=True)[:3]
    for match_id, score in top3:
        top3_list.append({
            'rfq_id': row_a['id'],
            'match_id': match_id,
            'similarity_score': score
        })

# Convert to DataFrame and save
top3_df = pd.DataFrame(top3_list)
top3_df.to_csv("../outputs/top3.csv", index=False, encoding='utf-8-sig')
display(top3_df.head())

Processing RFQs:   0%|                                                                        | 0/1005 [00:00<?, ?it/s]

Processing RFQs:   0%|                                                                | 1/1005 [00:01<22:43,  1.36s/it]

Processing RFQs:   0%|▏                                                               | 2/1005 [00:02<19:26,  1.16s/it]

Processing RFQs:   0%|▏                                                               | 3/1005 [00:03<17:13,  1.03s/it]

Processing RFQs:   0%|▎                                                               | 4/1005 [00:03<15:03,  1.11it/s]

...

Processing RFQs: 100%|████████████████████████████████████████████████████████████▉| 1003/1005 [15:07<00:01,  1.17it/s]

Processing RFQs: 100%|████████████████████████████████████████████████████████████▉| 1004/1005 [15:08<00:00,  1.12it/s]

Processing RFQs: 100%|█████████████████████████████████████████████████████████████| 1005/1005 [15:09<00:00,  1.10it/s]

Processing RFQs: 100%|█████████████████████████████████████████████████████████████| 1005/1005 [15:09<00:00,  1.10it/s]

,rfq_id,match_id,similarity_score
0,8aff426d-b8c0-43aa-ad26-835ef4de6129,2e56f82b-a80d-4704-83a5-14a52a117b65,0.600000
1,8aff426d-b8c0-43aa-ad26-835ef4de6129,7d1ab305-7fc6-4ab0-bc2a-9ae1e038e67e,0.544795
2,8aff426d-b8c0-43aa-ad26-835ef4de6129,9195d6ad-120f-4ae2-8e69-462ca56c3fc0,0.475000
3,37e624be-b125-464f-85b6-1838530193ef,56d8f8e0-7e07-42f7-9503-360aed60100a,0.200000
4,37e624be-b125-464f-85b6-1838530193ef,74fcfbc9-2569-46c8-ad3f-a9ea8b9f5a45,0.200000


In [1]:
readme_content = """
# Task B — RFQ Similarity

Hi! This is my submission for Task B, where I worked on finding similar RFQs (Request for Quotations) using the reference properties dataset. I also included some bonus analyses like alternative metrics, ablation analysis, and clustering. Below, I explain what I did step by step, why I did it this way, and what we learned.

---

## **Step 1: Load and Inspect Data**

I loaded `rfq.csv` and `reference_properties.tsv` and checked the columns.  

- This helps me understand what features are available.  
- I could see which features are numeric, categorical, or have missing values.  

**Outcome:** I know the structure of the data and which columns to process next.

---

## **Step B.1 — Reference Join & Missing Values**

- I normalized RFQ grades and reference grades (lowercase, stripped spaces) to match exactly.  
- I parsed numeric ranges like `2.5-3.0` or `≤5` into min, max, and mid values.  
- RFQs were merged with reference data on normalized grades.  
- Missing references were flagged to know which RFQs don’t have reference properties.  

**Why:** Normalization ensures matching; parsing numeric ranges lets us compute numeric similarity.  
**Assumptions:** If max value is missing, I set max = min.  
**Outcome:** RFQs now have corresponding reference data with missing ones flagged.

---

## **Step B.2 — Feature Engineering**

- Created min, max, and mid columns for numeric dimensions and grades.  
- Filled missing max values with min.  
- Kept categorical features like `coating`, `finish`, `form`, `surface_type`.  

**Why:** This allows numeric similarity computations and categorical comparisons.  
**Outcome:** RFQs have clean numeric and categorical features ready for similarity calculations.

---

## **Step B.3 — Similarity Calculation**

- **Numeric similarity:** Computed **Interval IoU** for each numeric dimension.  
- **Categorical similarity:** 1 if values match, 0 if different, averaged across all categories.  
- **Grade similarity:** Ratio of smaller/larger for numeric grade properties.  
- Combined all similarities using weighted average: dimensions 0.5, categorical 0.2, grades 0.3.  
- Extracted **top-3 most similar RFQs** per RFQ.  

**Why:** Weighted combination captures all aspects of similarity.  
**Assumptions:** Equal importance per numeric dimension, exact match for categorical features.  
**Outcome:** `top3.csv` with RFQ ID, top-3 matches, and similarity scores.

---

## **Step B.4 — Alternative Metrics & Ablation Analysis (Bonus)**

- Used **cosine similarity** for numeric features and **Jaccard similarity** for categorical features.  
- Weighted combination of dimensions, grades, and categorical features.  
- Ablation analysis: tested dropping one feature group at a time to see the impact.  

**Why:** To check if other metrics improve similarity and which features contribute most.  
**Outcome:** `top3_bonus.csv` and plots showing feature impact.

---

## **Step B.5 — Clustering RFQs (Bonus)**

- Standardized numeric features and combined with one-hot encoded categorical features.  
- Used **KMeans** to cluster RFQs into 5 clusters.  
- Visualized clusters using **PCA** in 2D.  
- Saved `rfq_with_clusters.csv` and `pca_coordinates.csv`.  

**Why:** Helps understand structure and group similar RFQs together.  

---

## **Assumptions & Issues Faced**

- Missing numeric values: filled with 0 for similarity or min = max.  
- Some reference columns missing in the dataset; handled dynamically.  
- Performance: iterating over all RFQs is slow; used `tqdm` to monitor progress.  

---

## **Outputs**

- `top3.csv` — Top-3 most similar RFQs (Task B).  
- `top3_bonus.csv` — Top-3 using alternative metrics.  
- `rfq_with_clusters.csv` — RFQs with cluster assignments.  
- `pca_coordinates.csv` — 2D PCA coordinates for plotting clusters.  

---

## **How to Run the Pipeline**

- Everything is in `TaskB_notebook.ipynb` and can be executed end-to-end.  
- Alternatively, use `run.py` to execute the full pipeline in one go.  
- All outputs will be saved in the `outputs/` folder.

# Setup and Run Instructions
1. Python Environment

These notebooks are written in Python 3.

It is recommended to use Conda to manage your environment and dependencies.

## To create a new environment, you can run:

conda create -n vanillasteel python=3.10
conda activate vanillasteel


## Install required packages using pip or conda. Here are the main libraries used in the notebooks:

pandas

numpy

openpyxl

matplotlib

seaborn

scikit-learn

re (regular expressions)

## Any other standard Python libraries

You can install them with:

pip install pandas numpy openpyxl matplotlib seaborn scikit-learn

## 2. Folder Structure

Make sure your project folder is organized like this:

vanillasteel-assessment/
│
├── notebooks/          # All Task A, Task B, Bonus Task notebooks
├── data/               # Supplier files and any other input data
├── outputs/            # Folder to save processed CSV files
├── README-taskA.txt
├── README-taskB.txt
├── bonus_task_output/  # Folder to save outputs from the bonus task
├── README-Bonus.md


The data folder contains supplier files (e.g., supplier_data1.xlsx, supplier_data2.xlsx) needed for Task A.

The outputs folder is where the CSV files generated by the notebooks (e.g., inventory_dataset.csv, top3.csv) will be saved. Ensure this folder already exists.

## 3. Running the Notebooks
## Task B

Task B is in a separate notebook.

Open the Task B notebook and run cells sequentially.

The output file top3.csv will be saved in the outputs folder.

Task B does not depend on the outputs of Task A.

Note: CSV files are saved using UTF-8 with BOM (utf-8-sig) encoding so that special characters like German letters (ü, ö, ß) display correctly in Excel.
## 4. Notes on CSV and Excel

If opening CSV files in Excel, make sure to choose UTF-8 encoding to view German characters correctly.

In Excel:

Open Excel.

Go to Data → Get Data → From Text/CSV.

Select the file and set File Origin to UTF-8.

This ensures columns like description and finish display correctly without garbled text.

## 5. Optional

If using Jupyter Notebook, make sure your kernel is set to the Python environment you installed with the required libraries.

Always check the outputs folder to ensure CSV files are generated correctly after running the notebooks.
"""


# Save to README.md
import os

# Folder where bonus outputs are saved
output_folder = "../outputs"
readme_path = os.path.join(output_folder, "README-Task B.md")
with open(readme_path, "w", encoding="utf-8") as f:
    f.write(readme_content)
print(f"README.md successfully saved at: {readme_path}")

README.md successfully saved at: ../outputs\README-Task B.md
